In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('/opt/slh/icecube/')

import os
os.environ["CUDA_VISIBLE_DEVICES"]="1,0"
#os.environ["CUDA_VISIBLE_DEVICES"]="1"
#os.environ["NCCL_P2P_DISABLE"] = "1"

In [ ]:
import polars as pl
import pandas as pd
import gc
import os
import numpy as np
from icecube.fastai_fix import *
from tqdm.notebook import tqdm
from icecube.data_train import RandomChunkSampler,LenMatchBatchSampler,IceCubeCache, DeviceDataLoader
from icecube.loss import loss, loss_vms
from icecube.models import EncoderWithDirectionReconstructionV18
from fastxtend.vision.all import EMACallback

In [ ]:
OUT = 'V18FT2'
PATH = '../data/'

NUM_WORKERS = 24
SEED = 2023
bs = 1024-256
L = 196

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)
os.makedirs(OUT, exist_ok=True)

In [ ]:
def WrapperAdamW(param_groups,**kwargs):
    return OptimWrapper(param_groups,torch.optim.AdamW)



In [ ]:
fname = OUT

ds_train = IceCubeCache(PATH, mode='train', L=L, reduce_size=0.125)
ds_train_len = IceCubeCache(PATH, mode='train', L=L, reduce_size=0.125, mask_only=True)
sampler_train = RandomChunkSampler(ds_train_len, chunks=ds_train.chunks)
len_sampler_train = LenMatchBatchSampler(sampler_train, batch_size=bs, drop_last=True)
dl_train = DeviceDataLoader(torch.utils.data.DataLoader(ds_train, 
            batch_sampler=len_sampler_train, num_workers=4, persistent_workers=True))

ds_val = IceCubeCache(PATH, mode='eval', L=L)
ds_val_len = IceCubeCache(PATH, mode='eval', L=L, mask_only=True)
sampler_val = torch.utils.data.SequentialSampler(ds_val_len)
len_sampler_val = LenMatchBatchSampler(sampler_val, batch_size=bs, drop_last=False)
dl_val= DeviceDataLoader(torch.utils.data.DataLoader(ds_val, batch_sampler=len_sampler_val,
            num_workers=0))

data = DataLoaders(dl_train,dl_val)
model = EncoderWithDirectionReconstructionV18()
model.load_state_dict(torch.load('/opt/slh/icecube/hb_training_loop/V18FT2/models/model_0.pth'))
model = nn.DataParallel(model)
model = model.cuda()
learn = Learner(data, model,  path = OUT, loss_func=loss_vms,cbs=[GradientClip(3.0),CSVLogger(),EMACallback(),
            SaveModelCallback(monitor='loss',comp=np.less,every_epoch=True),
            GradientAccumulation(n_acc=4096//bs)],
            metrics=[loss], opt_func=partial(WrapperAdamW,eps=1e-7)).to_fp16()




In [ ]:
learn.fit_one_cycle(8, 
                    lr_max=1e-5,
                    wd=0.05, 
                    pct_start=0.01, 
                    div=25,
                    div_final=25,
                    moms=(0.95,0.95,0.95))

In [ ]:
,